## 1. 填写账号密码

In [ ]:
import requests

def login():
    
    username = ""
    password = ""
    s = requests.Session()
    s.auth = (username, password)
    response = s.post('https://api.worldquantbrain.com/authentication')
    print(response.content)
    return s 

## 2.一些函数的定义

In [ ]:
from os import environ
from time import sleep
import time
import json
import pandas as pd
import random
import pickle
from itertools import product
from itertools import combinations
from collections import defaultdict
from urllib.parse import urljoin
import pickle
import requests
import json
from os.path import expanduser
from requests.auth import HTTPBasicAuth
from datetime import datetime, timezone, timedelta
def get_latest_monday(tz=None):
    if tz is None:
        tz = timezone(timedelta(hours=-4))
    
    now = datetime.now(tz)
    days_since_monday = now.weekday()
    monday = now - timedelta(days=days_since_monday)
    monday_start = monday.replace(hour=0, minute=0, second=0, microsecond=0)
    return monday_start.isoformat(timespec='minutes')

reverse_super_alpha_list = []
def super_score(rec):
    # 一个粗略计算super_alpha得分的函数,权重可调整
    sharpe = rec[1]**3
    turnover = rec[2]
    fitness = rec[3]**3
    margin = rec[4]
    if(rec[5]==None):
        pc=9999999
    else:
        pc=rec[6]*1000000
    return sharpe * (1 - turnover) *fitness *  margin -pc

def check_prod(alpha_bag,start):
    s = login()
    for idx, i in enumerate(alpha_bag):
        if idx < start:
            continue
        if idx % 100 == 0:
            print(idx)
        if idx % 200 == 0 & idx!=0 :
            s = login()
        #print(idx)
        pc = get_check_prod(s, i[0])
        i.append(pc)
        alpha_bag[idx] = i
        print(i)
        print(f"Processed {idx} alphas")


def get_check_prod(s, alpha_id):
    while True:
        result = s.get(f"https://api.worldquantbrain.com/alphas/{alpha_id}/correlations/prod")
        if "retry-after" in result.headers:
            time.sleep(float(result.headers["Retry-After"]))
        elif result.status_code == 429:
            print("Rate limit exceeded, sleeping for 60 seconds")
            time.sleep(60)
            continue
        else:
                try:
                    r = result.json()
                    print({alpha_id: r['max']})
                    return r['max']
                except Exception as e:
                    print(f"Error for alpha_id: {alpha_id}, {e}")
                return

def score(rec):
    # 一个粗略计算alpha得分的函数,权重可调整
    sharpe = rec[1]*5
    turnover = rec[2]
    fitness = rec[3]*5
    margin = rec[4]*2
    return sharpe * (1 - turnover) *fitness *  margin 
def verfiy_result(res):
    checks = res['is']['checks']
    flag = True
    for check in checks:
        if check['result'] in ['FAIL', 'ERROR']:
            flag = False
            if(check['name'] == "LOW_SHARPE"):
                reverse_super_alpha_list.append([res['id'],res['is']['sharpe'],res['is']['fitness'],res['is']['margin']])
            print("IS failed:", res['id'] , check['name'], check['result'])
            break
    return flag
def sus(start, end, sharpe_th, fitness_th,alpha_num, usage):
    s = login()
    output = []
    # 3E large 3C less
    count = 0
    for i in range(0, alpha_num, 100):
        url_e = "https://api.worldquantbrain.com/users/self/alphas?limit=100&offset=%d"%(i) \
                + "&status=UNSUBMITTED%1FIS_FAIL&dateCreated%3E=" + start  \
                + "&dateCreated%3C" + end \
                + "&is.fitness%3E" + str(fitness_th) + "&is.sharpe%3E" \
                + str(sharpe_th) + "&order=-is.sharpe&hidden=false&type=SUPER"
        url_c = "https://api.worldquantbrain.com/users/self/alphas?limit=100&offset=%d"%(i) \
                + "&status=UNSUBMITTED%1FIS_FAIL&dateCreated%3E=" + start  \
                + "&dateCreated%3C" + end \
                + "&is.fitness%3C-" + str(fitness_th) + "&is.sharpe%3C-" \
                + str(sharpe_th) +  "&order=is.sharpe&hidden=false&type=SUPER"
        urls = [url_e]
        if usage != "submit":
            urls.append(url_c)
        for url in urls:
            response = s.get(url)
            #print(response.json())
            try:
                alpha_list = response.json()["results"]
                #print(response.json())
                
                for j in range(len(alpha_list)):
                    if not verfiy_result(alpha_list[j]):
                        continue
                    alpha_id = alpha_list[j]["id"]
                    name = alpha_list[j]["name"]
                    dateCreated = alpha_list[j]["dateCreated"]
                    sharpe = alpha_list[j]["is"]["sharpe"]
                    fitness = alpha_list[j]["is"]["fitness"]
                    turnover = alpha_list[j]["is"]["turnover"]
                    margin = alpha_list[j]["is"]["margin"]
                    longCount = alpha_list[j]["is"]["longCount"]
                    shortCount = alpha_list[j]["is"]["shortCount"]
                    decay = alpha_list[j]["settings"]["decay"]
                    count += 1
                    #if (sharpe > 1.2 and sharpe < 1.6) or (sharpe < -1.2 and sharpe > -1.6):
                    if (longCount + shortCount) > 100:
                        rec = [alpha_id,sharpe, turnover, fitness, margin, decay]
                        print(rec)
                        if turnover > 0.7:
                            rec.append(decay*4)
                        elif turnover > 0.6:
                            rec.append(decay*3+3)
                        elif turnover > 0.5:
                            rec.append(decay*3)
                        elif turnover > 0.4:
                            rec.append(decay*2)
                        elif turnover > 0.35:
                            rec.append(decay+4)
                        elif turnover > 0.3:
                            rec.append(decay+2)
                        output.append(rec)
                        
            except Exception as e:
                print(f"Error in response for url: {url}")
                print(f"Exception type: {type(e)}")
                error_check=response
                print(f"{i} finished re-login")
                s = login()
    print("count: %d"%count)
    output.sort(key=score, reverse=True)
    return output  

         

## 3.筛选本周super alpha

- 通过datatime函数 获取本周开始时间与现在时间作为alpha筛选范围(每周不同alpha副本)
- 便于修改的筛选条件
- 自动过滤IS failed alpha 并给出failed:原因
- 对于shapre,fitness绝对值大小符合筛选条件,符号为负的alpha,单独列出,以待修改


In [ ]:

tz_ny = timezone(timedelta(hours=-4))
start = get_latest_monday(tz_ny) 
end = datetime.now(tz_ny).isoformat(timespec='minutes')

print(f"开始时间(默认本周一开始时间): {start}")
print(f"结束时间: {end}")

In [ ]:
'''
起始时间 结束时间 shapre fitness
对于开始时间想要细调,也可以换成
super_list=sus("2025-06-17T06:06-04:00",end,5,5,300, "track")
这样的形式 来细调
避免反复check 某些alpha的prod corr
'''
super_list=sus(start,end,5,5,300, "track")
super_list

In [ ]:
for i in reverse_super_alpha_list:
    print(i)

## 4.对筛选出的alpha进行prod corr检查

- 修改保存在输入的super_list全局变量中,**支持断点继续**

- 对于none prod corr再获取

In [ ]:
#最后一个参数为开始位置，可断点继续
check_prod(super_list,0)

In [ ]:
cnt_not_none=0
cnt_len=0
jishu=[]
none_list=[]
for i in super_list:
    if(len(i)>=10):
        cnt_len+=1
        if(i[9] is not None):
            cnt_not_none+=1
        else:
            jishu.append(cnt_len-1)
            none_list.append(i)
print(f"Total alphas with length >= 10: {cnt_len}, Alphas with non-None prod check: {cnt_not_none}")


result_bag=check_prod(none_list,0 )

for i in range(len(none_list)):
    if(none_list[i][0]==super_list[jishu[i]][0]):
        super_list[jishu[i]]=none_list[i]

## 5.排序输出


In [ ]:
super_list.sort(key=super_score, reverse=True)
super_list 

In [ ]:
flag= True
for item in super_list:
    print(item)
    if (item[-1]>0.7)&flag:
        flag=False
        print("-------------------除了正上方那一个以上均为prod<0.7------------------------")
